# Calculating phase misclosure

**Author**: Robert Zinke - Jet Propulsion Laboratory

This notebook demonstrates phase triplet misclosure calculation using the **`ariaMisclosure.py`** function. In this tutorial, we will introduce the concept of phase misclosure as a means of assessing the quality of an interferogram data set. We will then show how the `ariaMisclosure.py` tool can be used to explore the spatial and temporal patterns of misclosure in a data set.

## Table of Contents

[**What is phase misclosure**](#what_is_phase_misclosure)

[**Misclosure computation**](#misclosure_computation)
- [Loading data](#loading_data)
- [Phase triplet formulation](#phase_triplet_formulation)
- [Reference point](#reference_point)

[**Misclosure analysis**](#misclosure_analysis)
- [Net misclosure and absolute misclosure](#net_misclosure_and_absolute_misclosure)
- [Misclosure visualization](#misclosure_visualization)

[**Use in timeseries analysis**](#use_in_timeseries_analysis)

## Prep A. Initial setup of the notebook

If you have not already done so, it is recommended that you first check out the [**ariaTSsetup_tutorial**](https://github.com/aria-tools/ARIA-tools-docs/blob/master/JupyterDocs/ariaTSsetup/ariaTSsetup_tutorial.ipynb). The `ariaMisclosure.py` routines are designed to work with outputs produced by `ariaTSsetup.py`. Note that completion of ariaTSsetup_tutorial is not required, as this tutorial will use a different, smaller data set.

We first set up the file structure and initial notebook parameters. This step should be run each time this tutorial is (re)opened.

In [1]:
## Import standard modules
import os
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt

## Import ARIA-tools modules
from ARIAtools.computeMisclosure import stack

## Define the home and data directories at the processing location
work_dir = os.path.abspath(os.getcwd())
tutorial_home_dir = os.path.abspath(os.getcwd())
print("Work directory: ", work_dir)
print("Tutorial directory: ", tutorial_home_dir)

# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# Verify if ARIA-tools is installed correctly
try:
    import ARIAtools.shapefile_util as shputil
except:
    raise Exception('ARIA-tools is missing from your PYTHONPATH')
        
os.chdir(work_dir)

Work directory:  /Users/buzzanga/Software_InSAR/ARIA-tools-docs_git/JupyterDocs/ariaMisclosure
Tutorial directory:  /Users/buzzanga/Software_InSAR/ARIA-tools-docs_git/JupyterDocs/ariaMisclosure


## Prep B. Download and process the data

Next, we download the products using the `ariaDownload.py` function. For this tutorial, we will use data from descending track 42 over central California. This will take a few minutes if the products are not already downloaded. 
<div class="alert alert-warning">
  
<b>Login Credentials:</b>
Save your user-name and password to a `.netrc` file in your `$HOME` directory, or pass the combination explicitly using `ariaDownload.py --user <user> --pass <pass>`.
</div>

In [2]:
!ariaDownload.py -t 42 -b '35.5 37.5 -122 -120' -w products -s 20170701 -e 20180101

ARIA-tools Version: 1.1.0
Wrote -- 99 -- products to: /Users/buzzanga/Software_InSAR/ARIA-tools-docs_git/JupyterDocs/ariaMisclosure/products


<img src="./support_docs/Parkfield_overview.png" alt="track" width="600">

<blockquote>Google Earth image of the creeping section of the San Andreas Fault near Parkfield. Gray boxes are Track 42 acquisitions. White boxes denote are the bounding box of our study area. Faults from USGS Quaternary fault catalog are plotted in background. For interpretation of fault trace colors, refer to see <a> href="https://earthquake.usgs.gov/hazards/qfaults/">USGS fault catalogue website </a>. </blockquote>

<div class="alert alert-warning">
<b>Note:</b> Before proceeding, you must have an API key stored at ~/.topoapi on the same computer that is running this notebook. 
</div>

Follow instructions listed [here](https://opentopography.org/blog/introducing-api-keys-access-opentopography-global-datasets) to generate and access API key through OpenTopography.

```
echo "myAPIkey" > ~/.topoapi
chmod 600 ~/.topoapi
```
where myAPIkey is the key you just created.

We next extract the products for timeseries computation using `ariaTSsetup.py`. Extracted products for timeseries setup will go to the TSsetup subfolder.

In [3]:
!ariaTSsetup.py -w TSsetup -f 'products/*.nc' -d download -m download -b '35.5 37.5 -122 -120' -nt 4

ARIA-tools Version: 1.1.0
*****************************************************************
*** Time-series Preparation Function ***
*****************************************************************
Shapefile TSsetup/user_bbox.json created for input user bounds.
Multi-core version
All (99) GUNW products meet spatial bbox criteria.
Group GUNW products into spatiotemporally continuous interferograms.
All (33) interferograms are spatially continuous.
Thread count specified for gdal multiprocessing = 4
Download/cropping DEM
Applied cutline to produce 3 arc-sec SRTM DEM: TSsetup/DEM/SRTM_3arcsec.dem
***Downloading water mask... ***

Extracting unwrapped phase, coherence, and connected components for each interferogram pair
Generating: unwrappedPhase - [=>                       3%                       ] 20170724_20170712    0s /     0sNETCDF:"/Users/buzzanga/Software_InSAR/ARIA-tools-docs_git/JupyterDocs/ariaMisclosure/products/S1-GUNW-D-R-042-tops-20170724_20170712-140751-38619N_36645N-PP-

## What is phase misclosure
<a id= 'what_is_phase_misclosure'></a>

**Phase misclosure** is the disagreement between measurements of differential phase between common epochs *i*, *j*, and *k*. For the triplet of interferograms describing phase change $\Delta\phi_{ij}$, $\Delta\phi_{jk}$, $\Delta\phi_{ik}$, phase misclosure $\omega$ is simply the difference between the expected phase change from the sum $(\Delta\phi_{ij}$ + $\Delta\phi_{jk})$ and the phase difference measured by $\Delta\phi_{ik}$:

<div align = center> $\omega = \Delta\phi_{ij} + \Delta\phi_{jk} - \Delta\phi_{ik}$ <div /align = center>

This can be illustrated graphically with the following diagram.

<img src="./support_docs/PhaseClosure_diagram.png" alt="track" width="600">

<blockquote>Phase triplet misclosure is the disagreement between measurements with different time paths.</blockquote>

## Misclosure computation
<a id = 'misclosure_computation'></a>

Inputs for the `ariaMisclosure.py` function can be shown using the `-h` option. Examples are also shown.

In [4]:
## ariaMisclosure help menu
!ariaMisclosure.py -h

ARIA-tools Version: 1.1.0
usage: ariaMisclosure.py [-h] -f IMGFILE [-w WORKDIR] [--startdate STARTDATE]
                         [--enddate ENDDATE] [--exclude-pairs EXCLUDEPAIRS]
                         [--plot-pairs] [--mintime MINTIME]
                         [--maxtime MAXTIME] [--print-triplets]
                         [--plot-triplets] [-refX REFX] [-refY REFY]
                         [-refLon REFLON] [-refLat REFLAT] [--queryX QUERYX]
                         [--queryY QUERYY] [--queryLon QUERYLON]
                         [--queryLat QUERYLAT] [-v] [--pctmin PCTMINCLIP]
                         [--pctmax PCTMAXCLIP] [--plot-time-intervals]

Compute the cumulative misclosure of phase triplets based on a set of interferograms
saved in the stack/unwrapStack.vrt data set. During triplet computation, values at a reference point
are removed from the interferograms prior to misclosure computation to account for abiguities in the
unwrapped phase that might arise during pairwise com

### Loading data
<a id = 'loading_data'></a>

The `ariaMisclosure.py` function requires the "stack" file generated by `ariaTSsetup.py`. The "stack/unwrapStack.vrt" file contains links (.VRT) to all the stitched, unwrapped interferograms.

Through the command line interface, call the function using:

`ariaMisclosure.py -f stack/unwrapStack.vrt`


<div class="alert alert-warning">
Although the ariaMisclosure.py function only requires the filename (-f unwrapStack.vrt), the resulting analyses depend greatly on the selection of triplets evaluated, and the reference point chosen. Triplet definitions and selection are discussed in detail in the <b>Phase triplet formulation</b> section below. Reference point selection is described in the <b>Reference point</b> section.
</div>

Below, we instantiate the underlying class directly for demonstration purposes.

In [6]:
## Load data using "stack" object
dataStack = stack('TSsetup/stack/unwrapStack.vrt')

# [print(pair[0].strftime('%Y%m%d')+'_'+pair[1].strftime('%Y%m%d')) for pair in dataStack.pairs];
# [print((pair[0]-pair[1]).days) for pair in dataStack.pairs];

AttributeError: 'stack' object has no attribute 'verbose'

> /Users/buzzanga/Software_InSAR/ARIA-tools_git/tools/ARIAtools/computeMisclosure.py(167)__init__()
    165 
    166         # Other
--> 167         if self.verbose: logger.setLevel(logging.DEBUG)
    168 
    169 

ipdb> q


### Available pairs

As we will see below, the available triplet combinations depend on the available pairs. For an intuitive overview of the pairing strategy, invoke the `--plot-pairs` option.

This can be done using:
`ariaMisclosure.py -f stack/unwrapStack.vrt --plot-pairs`

In [ ]:
## Plot pairs
dataStack.plotPairs()

In the figure above, the pairs are ordered by secondary date. Note how the pair intervals range from 12 days to 60 days, in increments of 12 days (12, 24, 36, 48, 60).

### Phase triplet formulation
<a id = 'phase_triplet_formulation'></a>

Misclosure computation depends on the selection of epochs that constitute the "triplet" of interferograms. For instance, selection of pairs with 6- and 12-day lags...

e.g., for dates *i* 20150101, *j* 20150107, and *k* 20150114

20150107_20150101 ( *j i* = 6 days)

20150113_20150107 ( *k j* = 6 days)

20150113_20150101 ( *k i* = 12 days)

... will lead to a different result than pairs with 12- and 24-day lags...

e.g., for dates *i* 20150101, *j* 20150113, and *k* 20150125

20150113_20150101 ( *j i* = 12 days)

20150125_20150113 ( *k j* = 12 days)

20150125_20150101 ( *k i* = 24 days)


### Triplet selection

It is left to the user to define the triplet strategy of choice, keeping in mind what pairs are available in the data set.

By default, `ariaMisclosure.py` will create a list of all possible triplets based on the dates provided. To view a list of all possible triplets, invoke the `--print-triplets` flag:

`ariaMisclosure.py -f stack/unwrapStack.vrt --print-triplets`

For a schematic of the triplets, use `--plot-triplets`:

`ariaMisclosure.py -f stack/unwrapStack.vrt --plot-triplets`

In [ ]:
## Create list of all possible triplets:
#    `ariaMisclosure.py -f stack/unwrapStack.vrt --plot-triplets`
dataStack.createTriplets()

## Plot triplet schematic
dataStack.plotTriplets()
plt.gca().set_title('ALL possible triplets');

To reduce the list to include only pairs and triplets of interest, `ariaMisclosure.py` provides several inputs options for the user.

For example, let's say that we only want to use pairs with 12-day and 24-day intervals. Use the `--maxtime` option to specify the maximum allowable time interval:

`ariaMisclosure.py -f stack/unwrapStack.vrt --plot-triplets --mintime 12 --maxtime 24`

In [ ]:
## Limit triplets to 12-24 days
dataStack.createTriplets(minTime=12,maxTime=24)

## Plot triplet schematic
dataStack.plotTriplets()
plt.gca().set_title('Triplets with pair intervals between 12 and 24 days');


These criteria result in perfectly symmetric pair durations for each triplet, but perhaps the selection criteria are too stringent, resulting in gaps. If we include 36-day pairs, broader temporal coverage is available.

In [ ]:
## Limit triplets to 12-36 days
dataStack.createTriplets(minTime=12,maxTime=36)

## Plot triplets
dataStack.plotTriplets()
plt.gca().set_title('Triplets with pairs between 12 and 36 days');

Note that this strategy, however, gives more than one triplet for a given start date. Interpretation of the results can be complicated in this case, so for this demonstration we will return to our maximum time limit of 24 days.

In [ ]:
## Reformulate triplets
dataStack.createTriplets(minTime=12,maxTime=24)

### Reference point
<a id = "reference_point"></a>

The interferograms generated through the ARIA workflow are processed in a pair-wise manner. That is, each secondary image is registered to its respective master; interferograms are not coregistered in a stack to a single master. A reference point in space is therefore necesary to align the interferograms such that their phases are meaningful, much in the same way that a spatial reference point is necessary for timeseries computation.

`ariaMisclosure.py` offsers several options for selecting the reference point:

- Automatic
- XY pixel specification
- Lon/Lat specification

By default, `ariaMisclosure.py` will automatically select a random reference point at a pixel with high average filtered coherence. Specifically, the routines call the filtered coherence maps from stack/cohStack.vrt. These maps are summed to find pixels with average coherence > 0.7, from which a random location is chosen. This can be used as a first pass approach, but it is strongly recommended that the user identify an optimal location and define the reference point manually.

Here, we use the reference point -120.96$^\circ$E, 36.61$^\circ$N, which is in an unfarmed field away from steep topography or major tectonic structures. The command for this is:

`ariaMisclosure.py -f stack/unwrapStack.vrt --mintime 12 --maxtime 24 -refLon -120.96 -refLat 36.61`

Using this function in the command line, misclosure computation is carried out automatically. Below, we call the computeMisclosure() funciton for demonstration purposes.

In [ ]:
## Compute misclosure
dataStack.computeMisclosure(refLoLa=[-120.96, 36.61])

As described [above](#what_is_phase_misclosure), misclosure is calculated by summing interferograms from common epochs *i*, *j*, *k*:


<div align = "center"> $\omega = \Delta\phi_{ji} + \Delta\phi_{kj} - \Delta\phi_{ki}$ </div>


where all three interferograms are adjusted to a common reference point.

This is illustrated below using a single triplet of interferograms:

20170724_20170712 , 20170805_20170724 , 2017805_20170712


<img src="./support_docs/MisclosureSum.png" alt="track" width="850">

<blockquote>Triplet of interferograms. Black square is the reference point.</blockquote>

The resulting map of misclosure from that triplet is shown:

<img src="./support_docs/Misclosure.png" width="500">

<blockquote>Misclosure from triplet. Black square is the reference point&mdash;here, misclosure is by definition zero.</blockquote>

In the above misclosure map, note how misclosure is larger (in an absolute sense) in the agricultural areas.

<div class="alert alert-warning">
By definition, misclosure will be zero at the reference point. Similarly, the spatial patterns of misclosure might change if the reference point is relocated, especially if different connected components are present at the reference point location.
</div>

## Misclosure analysis
<a id = "misclosure_analysis"></a>

### Net misclosure and absolute misclosure
<a id = "net_misclosure_and_absolute_misclosure"></a>

Two values of misclosure are calculated: (1) the **net misclosure**; and (2) the **absolute misclosure**.

- The net misclosure is simply the sum of all misclosure measurements (e.g., $\omega$ = -0.3 + 0.2 + 0.1 = 0.0). The range of values can therefore be positive or negative.

- The absolute misclosure uses the absolute value of misclosure measurements (e.g., |$\omega$| = |-0.3| + |0.2| + |0.1| = 0.6). Absolute misclosure values are therefore only positive.

Both misclosure and absolute misclosure are stored as multi-dimensional arrays, with each "slice" representing the misclosure map associated with a triplet.

### Misclosure visualization
<a id = "misclosure_visualization"></a>

**Misclosure for each triplet**

The misclosure maps for each triplet of dates are printed as PNGs and stored in the "MisclosureFigs" folder.

**Cumulative misclosure**

For data sets of more than a dozen or so triplets, it is convenient to visualize the misclosure as a cumulative sum of all the misclosure maps. This highlights places where misclosure is consistently poor.

When running `ariaMisclosure.py`, three plots will automatically appear. These are:
- cumulative misclosure
- cumulative absolute misclosure
- a plot with four graphs that will show the time history of misclosure for the triplets

These three plots are interactive when run through the command line. Static versions of the plots are shown below. Here, we pre-specify a query point to view the misclosure series. The query point (-121.11$^\circ$, 36.51$^\circ$) is shown as a white dot on the left-hand side of the maps below. The net misclosure, cumulative net misclosure, absolute misclosure, and cumulative absolute misclosure are shown in the line chart at the bottom.

In [ ]:
## Visualize misclosure
dataStack.plotCumMisclosure(queryLoLa=[-121.11,36.51])

<div class="alert alert-warning">
<b>Note</b>, the time history of misclosure values are plotted as a function of the middle date of each triplet. If there is more than one triplet for each date, the time histories might be more difficult to interpret.
</div>

Using the command line tool, the use can click on different pixels of either the net misclosure or absolute misclosure maps to see the time history of any pixel. The pixel values are printed to the terminal.

Once the interactive plots are closed, the figures are saved to files.

The cumulative net and absolute misclosure figures are saved as georeferenced GeoTiffs for use with mapping software.

## Use in timeseries analysis
<a id = "use_in_timeseries_analysis"></a>

Triplet misclosure analysis can help determine what portion of your observed velocities may be due to measurement error from multilooking. If the misclosure computation shows a systematic trend (e.g., cumulative misclosure increases with time), then the apparent LOS velocities in your timeseries calculation will not represent true ground surface velocities. Various mitigation strategies for this problem can be found in the literature.

Furthermore, sudden jumps or increases/decreases in misclosure can indicate unwrapping errors that went unnoticed in the initial selection of interferograms. The user can decide whether those interferograms with unwrapping errors are appropriate for further timeseries analysis.